In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Random Descriptives") \
    .config("spark.jars", "/home/jovyan/work/gcs-connector-hadoop2-latest.jar") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/home/jovyan/work/key.json") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

In [54]:
# tweets = spark.read.parquet('gs://spain-tweets/rehydrated/tweets-2017-10-16')
month = '201806'
tweets = spark.read.parquet('gs://spain-tweets/rehydrated/lake').where(f'month = {month}')
# tweets = spark.read.parquet('gs://spain-tweets/rehydrated/tweets-2017-10-16')
tweets.createOrReplaceTempView('tweets')

In [ ]:
# 201806 -- 743146
# 201710 -- 615765
spark.sql('select rehydrated.id_str from tweets where rehydrated is not null').count()

In [13]:
# 201806 -- 404789
# 201710 -- 347435

spark.sql('select rehydrated.id_str from tweets where rehydrated.retweeted_status is not null').count()

347435

In [55]:
df = spark.sql('with t as (select rehydrated.id_str as id_str, rehydrated.user.id_str as user, struct(rehydrated.retweeted_status.text as text, rehydrated.retweeted_status.id_str, rehydrated.retweeted_status.user.id_str as user, rehydrated.retweeted_status.user.screen_name) as retweeted_status from tweets where rehydrated.retweeted_status is not null) select retweeted_status.id_str, retweeted_status, count(*) from t group by retweeted_status')

In [56]:
df.printSchema()

root
 |-- id_str: string (nullable = true)
 |-- retweeted_status: struct (nullable = false)
 |    |-- text: string (nullable = true)
 |    |-- id_str: string (nullable = true)
 |    |-- user: string (nullable = true)
 |    |-- screen_name: string (nullable = true)
 |-- count(1): long (nullable = false)



In [57]:
dat = df.toPandas()

In [58]:
def flatten(df, col, prefix):
    df = df.copy()
    keys = df[col][0].keys()
    for key in keys:
        df[f'{prefix}_{key}'] = df[col].map(lambda d: d[key])
    return df

In [59]:
# Take a look at most popular tweets for a given month

dat['retweeted_status'] = dat.retweeted_status.map(lambda x: x.asDict())

flatten(dat, 'retweeted_status', 'retweeted_status') \
    .drop(columns = ['id_str', 'retweeted_status']) \
    .rename(columns = {'count(1)': 'count'}) \
    .sort_values('count', ascending=False) \
    .head(100) \
    .to_csv(f'top-retweets-{month}.csv', index=False)

In [21]:
import pandas as pd
import numpy as np

def make_hist(col):
    counts, vals = np.histogram(col, bins=[0,2,4,8,16,32,64,128,256,512,1024,2048])
    return pd.DataFrame({'upper_value': vals[1:], 'count': counts})

dat = dat.rename(columns = {'count(1)': 'count'}).sort_values('count', ascending=False)

make_hist(dat['count'])

,upper_value,count
0,2,144251
1,4,21744
2,8,6710
3,16,2937
4,32,1449
5,64,617
6,128,201
7,256,47
8,512,10
9,1024,3
